In [1]:
# @title 🛠️ Install requirments
#!DEBIAN_FRONTEND=noninteractive
!sudo apt-get update # && sudo apt-get -y upgrade
!sudo apt-get -y install libegl1
!sudo apt-get -y install libopengl0
!sudo apt-get -y install libxcb-cursor0
!pip install -r https://raw.githubusercontent.com/daswer123/xtts-finetune-webui/main/requirements.txt
!pip install gradio==4.44.1
!pip install fastapi==0.103.1
!pip install pydantic==2.3.0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [1]:
import os
import torch
from TTS.api import TTS

# Step 1: Create model_files directory
os.makedirs('model_files', exist_ok=True)

# Step 2: Load the model using TTS API
print("Loading the model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=False, gpu=False)
print("Model loaded successfully.")

# Helper function to save the quantized model and print its size
def save_and_print_size(model, filename):
    model_path = f'model_files/{filename}.pth'
    torch.save({'model': model.state_dict(), 'config': tts.config}, model_path)
    size_mb = os.path.getsize(model_path) / 1e6  # Convert size to MB
    print(f"{filename} saved. Size: {size_mb:.2f} MB")

# Try different quantization techniques
print("Applying quantization techniques...")

# 1. Dynamic Quantization
try:
    print("Trying Dynamic Quantization...")
    quantized_dynamic = torch.quantization.quantize_dynamic(
        tts, {torch.nn.Linear, torch.nn.LSTM, torch.nn.GRU}, dtype=torch.qint8
    )
    save_and_print_size(quantized_dynamic, "quantized_dynamic")
except Exception as e:
    print(f"Dynamic Quantization failed: {e}")

# 2. Static Quantization (requires prepared model)
try:
    print("Trying Static Quantization...")
    tts.eval()  # Ensure the model is in evaluation mode
    tts_prepared = torch.quantization.prepare(tts, inplace=False)
    quantized_static = torch.quantization.convert(tts_prepared, inplace=False)
    save_and_print_size(quantized_static, "quantized_static")
except Exception as e:
    print(f"Static Quantization failed: {e}")

# 3. Weight-only Quantization (FP16)
try:
    print("Trying Weight-Only Quantization (FP16)...")
    tts_half = tts.half()  # Convert weights to half precision
    save_and_print_size(tts_half, "quantized_fp16")
except Exception as e:
    print(f"Weight-Only Quantization (FP16) failed: {e}")

# 4. Pruned Model Quantization (Optional)
try:
    print("Trying Pruned Quantization (Experimental)...")
    pruned_model = torch.nn.utils.prune.global_unstructured(
        tts, pruning_method=torch.nn.utils.prune.L1Unstructured, amount=0.2
    )
    quantized_pruned = torch.quantization.quantize_dynamic(
        pruned_model, {torch.nn.Linear, torch.nn.LSTM, torch.nn.GRU}, dtype=torch.qint8
    )
    save_and_print_size(quantized_pruned, "quantized_pruned")
except Exception as e:
    print(f"Pruned Quantization failed: {e}")

# Compare the original model size
original_model_path = os.path.join(
    torch.hub.get_dir(), "checkpoints", "tts_models--multilingual--multi-dataset--xtts_v2", "model_file.pth"
)
if os.path.exists(original_model_path):
    original_size = os.path.getsize(original_model_path) / 1e6
    print(f"Original model size: {original_size:.2f} MB")
else:
    print("Original model file not found.")


Loading the model...


/usr/local/lib/python3.10/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path)
/u

Model loaded successfully.
Applying quantization techniques...
Trying Dynamic Quantization...
quantized_dynamic saved. Size: 1776.12 MB
Trying Static Quantization...


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/quantize.py:320: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "


quantized_static saved. Size: 1868.13 MB
Trying Weight-Only Quantization (FP16)...
quantized_fp16 saved. Size: 934.33 MB
Trying Pruned Quantization (Experimental)...
Pruned Quantization failed: module 'torch.nn.utils' has no attribute 'prune'
Original model file not found.


In [1]:
import os
import torch
from TTS.api import TTS

# Step 1: Create model_files directory
os.makedirs('model_files', exist_ok=True)

# Step 2: Load the model using TTS API
print("Loading the model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=False, gpu=False)
print("Model loaded successfully.")

# Helper function to save the quantized model and print its size
def save_and_print_size(model, filename):
    model_path = f'model_files/{filename}.pth'
    torch.save({'model': model.state_dict(), 'config': tts.config}, model_path)
    size_mb = os.path.getsize(model_path) / 1e6  # Convert size to MB
    print(f"{filename} saved. Size: {size_mb:.2f} MB")

# Try different quantization techniques
print("Applying quantization techniques...")

# 1. Dynamic Quantization
try:
    print("Trying Dynamic Quantization...")
    quantized_dynamic = torch.quantization.quantize_dynamic(
        tts, {torch.nn.Linear, torch.nn.LSTM, torch.nn.GRU}, dtype=torch.qint8
    )
    save_and_print_size(quantized_dynamic, "quantized_dynamic")
except Exception as e:
    print(f"Dynamic Quantization failed: {e}")

# 2. Static Quantization (requires prepared model)
try:
    print("Trying Static Quantization...")
    tts.eval()  # Ensure the model is in evaluation mode
    tts_prepared = torch.quantization.prepare(tts, inplace=False)
    quantized_static = torch.quantization.convert(tts_prepared, inplace=False)
    save_and_print_size(quantized_static, "quantized_static")
except Exception as e:
    print(f"Static Quantization failed: {e}")

# 3. Weight-only Quantization (FP16)
try:
    print("Trying Weight-Only Quantization (FP16)...")
    tts_half = tts.half()  # Convert weights to half precision
    save_and_print_size(tts_half, "quantized_fp16")
except Exception as e:
    print(f"Weight-Only Quantization (FP16) failed: {e}")

# 4. Pruned Model Quantization (Optional)
import torch.nn.utils.prune as prune

try:
    print("Trying Pruned Quantization (Experimental)...")
    pruned_model = torch.nn.utils.prune.global_unstructured(
        tts, pruning_method=torch.nn.utils.prune.L1Unstructured, amount=0.2
    )
    quantized_pruned = torch.quantization.quantize_dynamic(
        pruned_model, {torch.nn.Linear, torch.nn.LSTM, torch.nn.GRU}, dtype=torch.qint8
    )
    save_and_print_size(quantized_pruned, "quantized_pruned")
except Exception as e:
    print(f"Pruned Quantization failed: {e}")

# Compare the original model size
original_model_path = os.path.join(
    torch.hub.get_dir(), "checkpoints", "tts_models--multilingual--multi-dataset--xtts_v2", "model_file.pth"
)
if os.path.exists(original_model_path):
    original_size = os.path.getsize(original_model_path) / 1e6
    print(f"Original model size: {original_size:.2f} MB")
else:
    print("Original model file not found.")


Loading the model...


/usr/local/lib/python3.10/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path)
/u

Model loaded successfully.
Applying quantization techniques...
Trying Dynamic Quantization...
quantized_dynamic saved. Size: 1776.12 MB
Trying Static Quantization...


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/quantize.py:320: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "


quantized_static saved. Size: 1868.13 MB
Trying Weight-Only Quantization (FP16)...
quantized_fp16 saved. Size: 934.33 MB
Trying Pruned Quantization (Experimental)...
Pruned Quantization failed: global_unstructured(): parameters is not an Iterable
Original model file not found.


In [2]:
# @title Use quantization for pruning

import os
import torch
from TTS.tts.models.xtts import Xtts
from TTS.tts.configs.xtts_config import XttsConfig
import torch.nn.utils.prune as prune
import urllib.request

# Step 1: Create the model_files directory
os.makedirs('model_files', exist_ok=True)

# Step 2: Download the model, config, and vocab files
print("Downloading model files...")
config_url = "https://huggingface.co/coqui/XTTS-v2/resolve/main/config.json?download=true"
model_url = "https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth?download=true"
vocab_url = "https://huggingface.co/coqui/XTTS-v2/resolve/main/vocab.json?download=true"

urllib.request.urlretrieve(config_url, 'model_files/config.json')
urllib.request.urlretrieve(model_url, 'model_files/model.pth')
urllib.request.urlretrieve(vocab_url, 'model_files/vocab.json')
print("Downloaded model files.")

# Step 3: Set device (CPU or GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 4: Load the XTTS model from checkpoint
print("Loading XTTS model...")
config_path = "model_files/config.json"
model_path = "model_files/model.pth"
vocab_path = "model_files/vocab.json"

config = XttsConfig()
config.load_json(config_path)

model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=model_path, vocab_path=vocab_path, use_deepspeed=False)
model.to(device)
print("XTTS model loaded successfully.")

# Step 5: Apply Pruning and Quantization
try:
    print("Applying Pruning and Quantization...")

    # Collect parameters to prune (Linear and Conv2d layers)
    parameters_to_prune = [
        (module, 'weight') for _, module in model.named_modules()
        if isinstance(module, (torch.nn.Linear, torch.nn.Conv2d))
    ]

    # Apply 20% global unstructured pruning
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=0.2,
    )

    # Remove pruning hooks to finalize the model
    for module, _ in parameters_to_prune:
        prune.remove(module, 'weight')

    # Apply dynamic quantization
    quantized_model = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear, torch.nn.LSTM, torch.nn.GRU}, dtype=torch.qint8
    )

    # Save the pruned and quantized model
    quantized_model_path = 'model_files/quantized_pruned_model.pth'
    torch.save({
        'model': quantized_model.state_dict(),
        'config': config.to_dict()
    }, quantized_model_path)

    # Compare model sizes
    original_size = os.path.getsize(model_path) / 1e6  # Convert size to MB
    quantized_size = os.path.getsize(quantized_model_path) / 1e6  # Convert size to MB

    print(f"Original Model Size: {original_size:.2f} MB")
    print(f"Quantized Pruned Model Size: {quantized_size:.2f} MB")

except Exception as e:
    print(f"Pruning and Quantization failed: {e}")


Downloaded model files.
Loading XTTS model...


/usr/local/lib/python3.10/dist-packages/trainer/io.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


XTTS model loaded successfully.
Applying Pruning and Quantization...
Original Model Size: 1867.93 MB
Quantized Pruned Model Size: 1776.06 MB


In [4]:
# @title ## Test quantized model
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Step 1: Set device (CPU or GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 2: Define paths for the config, model, and vocab files
config_path = "model_files/config.json"# @param {type:"string"}
model_path = "model_files/model.pth"# @param {type:"string"}
vocab_path = "model_files/vocab.json"# @param {type:"string"}
target_voice_path = "/content/ref.wav"  # @param {type:"string"}
output_path = "xtts_output.wav" # @param {type:"string"}
# Step 3: Load the XTTS Model from Checkpoint
print("Loading custom XTTS model...")
config = XttsConfig()
config.load_json(config_path)

# Initialize the XTTS model from the configuration
model = Xtts.init_from_config(config)

# Load the model checkpoint
model.load_checkpoint(config, checkpoint_path=model_path, vocab_path=vocab_path, use_deepspeed=False)

# Move the model to the appropriate device
model.to(device)
print("XTTS model loaded successfully.")

# Step 4: Compute Speaker Latents
print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[target_voice_path])
print("Speaker latents computed.")

# Step 5: Prepare Input Text for Inference
text = "Hello, world! This is a test of the XTTS model." # @param {type:"string"}
print(f"Input text: {text}")

# Step 6 Infrence model
print("Inference...")
out = model.inference(
    text,
    "en",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7, # Add custom parameters here
)
torchaudio.save(output_path, torch.tensor(out["wav"]).unsqueeze(0), 24000)

# Step 9: (Optional) Play the Audio if Running in Colab
try:
    from IPython.display import Audio
    display(Audio(output_path, rate=24000))  # Specify sample rate
except ImportError:
    print("IPython not available, skipping audio playback.")

Loading custom XTTS model...
XTTS model loaded successfully.
Computing speaker latents...
Speaker latents computed.
Input text: Hello, world! This is a test of the XTTS model.
Inference...


In [2]:
from TTS.api import TTS

# Initialize the TTS object with your model
print("Loading the model...")
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=False, gpu=False)
print("Model loaded successfully.")

# List all available attributes and methods
print("\nAvailable attributes and methods in the TTS object:")
for attr in dir(tts):
    print(attr)


Loading the model...
Model loaded successfully.

Available attributes and methods in the TTS object:
__all__
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__path__
__spec__
_named_member_accessor
clip_grad
clip_grad_norm
clip_grad_norm_
clip_grad_value_
convert_conv2d_weight_memory_format
convert_conv3d_weight_memory_format
convert_parameters
fuse_conv_bn_eval
fuse_conv_bn_weights
fuse_linear_bn_eval
fuse_linear_bn_weights
fusion
init
memory_format
parameters_to_vector
parametrizations
parametrize
remove_spectral_norm
remove_weight_norm
rnn
skip_init
spectral_norm
stateless
vector_to_parameters
weight_norm
